In [2]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Data
Split given training data into the following split:
- 60% Training set
- 20% Cross Validation Set
- 10% Test Set

In [12]:
df_all = pd.read_excel('train.xlsx')
df = df_all.drop(columns = ['Name', 'Cabin', 'Ticket'])

# all columns that will be used as features. Exclude target
features = [ x for x in df.columns if x not in 'Survived']


In [13]:
X_train, X_test, y_train, y_test = train_test_split(df[features], df['Survived'], train_size=0.8, random_state=42)
# display(X_train)
# display(X_test)
# display(y_train)
# display(y_test)